<a href="https://colab.research.google.com/github/funkdub/Pytorch/blob/master/LeNet_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print('Importing ...')
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets


import os
import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Importing ...


使用datasets 准备数据集，并且将训练集 验证机 以及 测试集准备好。

In [2]:
# 其实就是数据增强操作,在dataloader中使用transforms
data_transforms = transforms.Compose(
[
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
]
)

train_data = datasets.MNIST('data',train=True,download=True,transform=data_transforms)
test_data = datasets.MNIST('data',train=False,download=True,transform=data_transforms)

n_train = int(len(train_data)*0.9)
n_valid = len(train_data) - n_train

train_data,valid_data = torch.utils.data.random_split(train_data,[n_train,n_valid])

print('train_data num is',n_train,'|valid_data numm is ',n_valid,'|test_data numm is',len(test_data))

  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:00, 20094258.46it/s]                            


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz


32768it [00:00, 336799.09it/s]
0it [00:00, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 5829916.48it/s]                           
8192it [00:00, 130828.45it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!
train_data num is 54000 |valid_data numm is  6000 |test_data numm is 10000


参数

In [0]:
BATCH_SIZE = 64

train_data_it = data.DataLoader(train_data, shuffle=True, batch_size = BATCH_SIZE)
valid_data_it = data.DataLoader(valid_data, batch_size = BATCH_SIZE)
test_data_it = data.DataLoader(test_data, batch_size = BATCH_SIZE)


定义Model LeNet

In [0]:
class LeNet(nn.Module):
  def __init__(self):
    super(LeNet,self).__init__()
    self.conv1 = nn.Conv2d(1,6,5)
    self.conv2 = nn.Conv2d(6,16,5)
    self.fc1 = nn.Linear(5*5*16 , 120)
    self.fc2 = nn.Linear(120 , 84)
    self.fc3 = nn.Linear(84 , 10)
   
  def forward(self,x):
    x = F.max_pool2d(F.relu(self.conv1(x)) , 2)
    x = F.max_pool2d(F.relu(self.conv2(x)) , 2)
    x = x.view(x.shape[0],-1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = LeNet().to(device)
optimizer = optim.Adam(model.parameters())
loss_func = nn.CrossEntropyLoss()


cuda


计算准确率

In [0]:
def accuracy_cal(pred, y):
  # tensor.max选取 tensor中的最大值 
  # max(1)将 最大值的 index 作为结果输出
  prediction = pred.max(1)[1]
  correct = prediction.eq(y.view_as(prediction)).sum()
  acc = correct.float() / prediction.shape[0]
  return acc

In [0]:
def train(model, device , data , optimizer , loss_func):
  epoch_loss = 0
  epoch_acc = 0
  
  model.train()
  
  for (x,y) in data:
    
    x = x.to(device)
    y = y.to(device)

    optimizer.zero_grad()
    pred = model(x)
    loss = loss_func(pred, y)
    acc = accuracy_cal(pred,y)
    loss.backward()
    optimizer.step()
    
    # item()将单值tensor转换成标量
    epoch_loss += loss.item()
    epoch_acc += acc.item()
  return epoch_loss / len(data), epoch_acc / len(data)

In [0]:
def test(model,device,data,loss_func):
  epoch_loss = 0
  epoch_acc = 0
  model.eval()
  
  with torch.no_grad():
    for (x,y) in data:
      x = x.to(device)
      y = y.to(device)
      
      pred = model(x)
      loss = loss_func(pred,y)
      acc = accuracy_cal(pred,y)
      
      epoch_loss += loss.item()
      epoch_acc += acc.item()
  return epoch_loss / len(data), epoch_acc / len(data)
      

In [9]:
EPOCH = 10
SAVE_DIR = 'models'
MODEL_SAVE_DIR = os.path.join(SAVE_DIR,'LeNet_mnist.pt')

best_valid_loss = float('inf')


if not os.path.isdir(f'{SAVE_DIR}'):
  os.makedirs(f'{SAVE_DIR}')
  
for epoch in range(EPOCH):
  print('Epoch is ',epoch)
  train_loss , train_acc = train(model,device,train_data_it,optimizer,loss_func)
  valid_loss , valid_acc = test(model,device,valid_data_it,loss_func)
  
  if valid_loss < best_valid_loss:
    best_valid_loss = valid_loss
    torch.save(model.state_dict(),MODEL_SAVE_DIR)
  print('|EPOCH ',epoch,'|Train Loss is ',train_loss,'|Train Acc is ',train_acc,'|Valid Loss is ',valid_loss,'|Valid acc is ',valid_acc)

Epoch is  0
|EPOCH  0 |Train Loss is  0.25406545481834364 |Train Acc is  0.9211529917061612 |Valid Loss is  0.12763656691667882 |Valid acc is  0.9598847519844136
Epoch is  1
|EPOCH  1 |Train Loss is  0.0787083214475504 |Train Acc is  0.97540852095561 |Valid Loss is  0.07819700252978092 |Valid acc is  0.9753435285801583
Epoch is  2
|EPOCH  2 |Train Loss is  0.056325775496132 |Train Acc is  0.9827581951262262 |Valid Loss is  0.08136863386298114 |Valid acc is  0.9770057626227115
Epoch is  3
|EPOCH  3 |Train Loss is  0.043477747249483216 |Train Acc is  0.9862756714295436 |Valid Loss is  0.050501130164620725 |Valid acc is  0.9844858158142009
Epoch is  4
|EPOCH  4 |Train Loss is  0.03554896112769772 |Train Acc is  0.9882133689254381 |Valid Loss is  0.04967410662310555 |Valid acc is  0.9860372340425532
Epoch is  5
|EPOCH  5 |Train Loss is  0.029988751637471302 |Train Acc is  0.9903917357819905 |Valid Loss is  0.07298504759339576 |Valid acc is  0.9778368796439881
Epoch is  6
|EPOCH  6 |Train L

In [11]:
model.load_state_dict(torch.load(MODEL_SAVE_DIR))
test_loss, test_acc = test(model,device,test_data_it,loss_func)
print('Test Loss is ',test_loss,'|Test ACC is ',test_acc)

Test Loss is  0.032160650037086694 |Test ACC is  0.9917396496815286
